# Tennis Contact Point Analysis - MVP

Upload a **single image** of your contact moment to analyze body positioning and measure contact point spacing.

**Steps:**
1. Run the **Setup** cell to install dependencies
2. Run the **Upload & Analyze** cell — upload your contact frame image
3. View results: annotated image with skeleton overlay + measurements table

In [ ]:
#@title 1. Setup - Install dependencies and clone repo
import os, sys, shutil

REPO_URL = "https://github.com/xiaoxiang-ma/tennis_contact_point_spacing.git"
REPO_DIR = "/content/tennis_contact_point_spacing"

# Always re-clone to ensure latest code
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)
!git clone {REPO_URL} {REPO_DIR}

!pip install -q -r {REPO_DIR}/requirements.txt

# Clear any cached module imports from previous runs
for mod_name in list(sys.modules.keys()):
    if mod_name.startswith(("src.", "utils.")):
        del sys.modules[mod_name]

if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

print("Setup complete!")

In [ ]:
#@title 2. Upload & Analyze Contact Frame
import numpy as np
import pandas as pd
import cv2
from google.colab import files
from IPython.display import display, Image as IPImage
import os

from utils.coordinate_transforms import (
    pelvis_origin_transform, estimate_ground_plane,
    apply_ground_plane
)
from src.pose_estimation import PoseEstimator
from src.measurements import compute_measurements
from src.visualization import (
    annotate_contact_frame, save_annotated_frame
)

# --- Upload image ---
print("Upload your contact frame image (PNG, JPG):")
uploaded = files.upload()
image_filename = list(uploaded.keys())[0]
image_path = os.path.join("/content", image_filename)
with open(image_path, "wb") as f:
    f.write(uploaded[image_filename])

# --- Load image ---
print(f"\nLoading image: {image_filename}")
frame = cv2.imread(image_path)
if frame is None:
    raise ValueError(f"Could not load image: {image_path}")
h, w = frame.shape[:2]
print(f"  Resolution: {w}x{h}")

# --- Pose estimation ---
print("\nEstimating pose...")
pose_estimator = PoseEstimator(static_image_mode=True, model_complexity=2)
landmarks, raw_result = pose_estimator.process_frame(frame)

if landmarks is None:
    pose_estimator.close()
    raise ValueError("No pose detected in image. Make sure the player is clearly visible.")

pixel_lm = pose_estimator.get_pixel_landmarks(raw_result, frame.shape)
pose_estimator.close()
print("  Pose detected successfully!")

# --- Determine contact wrist (the one further extended from pelvis) ---
lw = landmarks.get("left_wrist", np.zeros(3))
rw = landmarks.get("right_wrist", np.zeros(3))
pelvis = landmarks.get("pelvis", np.zeros(3))

if np.linalg.norm(rw - pelvis) > np.linalg.norm(lw - pelvis):
    contact_wrist_name = "right_wrist"
    contact_point = rw
    print("  Detected: RIGHT-handed shot (right wrist further extended)")
else:
    contact_wrist_name = "left_wrist"
    contact_point = lw
    print("  Detected: LEFT-handed shot (left wrist further extended)")

# --- Transform coordinates ---
centered = pelvis_origin_transform(landmarks)
ground_z = estimate_ground_plane(centered)
adjusted = apply_ground_plane(centered, ground_z)

# Contact point in same coordinate system
contact_adjusted = contact_point - pelvis - np.array([0, 0, ground_z])

# --- Compute measurements ---
print("\nComputing measurements...")
meas = compute_measurements(adjusted, contact_adjusted)
meas["contact_wrist"] = contact_wrist_name

# --- Create output ---
output_dir = "/content/output"
os.makedirs(output_dir, exist_ok=True)

# Annotate frame
annotated = annotate_contact_frame(
    frame, pixel_lm, contact_wrist_name,
    meas, frame_num=0, fps=1.0
)
out_image_path = os.path.join(output_dir, f"contact_analyzed_{os.path.splitext(image_filename)[0]}.png")
save_annotated_frame(annotated, out_image_path)

# Save measurements CSV
csv_path = os.path.join(output_dir, f"measurements_{os.path.splitext(image_filename)[0]}.csv")
df = pd.DataFrame([meas])
df.to_csv(csv_path, index=False)

print(f"\nResults saved to {output_dir}/")
print("\n" + "="*50)
print("CONTACT POINT MEASUREMENTS")
print("="*50)
print(f"\nLateral offset:     {meas.get('lateral_offset_cm', 0):>7.1f} cm  ({meas.get('lateral_offset_inches', 0):>5.1f} in)")
print(f"Forward/back:       {meas.get('forward_back_cm', 0):>7.1f} cm  ({meas.get('forward_back_inches', 0):>5.1f} in)")
print(f"Height above ground:{meas.get('height_above_ground_cm', 0):>7.1f} cm  ({meas.get('height_above_ground_inches', 0):>5.1f} in)")
if "shoulder_line_distance_cm" in meas:
    print(f"Shoulder line dist: {meas.get('shoulder_line_distance_cm', 0):>7.1f} cm  ({meas.get('shoulder_line_distance_inches', 0):>5.1f} in)")
if "relative_to_shoulder_height_cm" in meas:
    print(f"vs Shoulder height: {meas.get('relative_to_shoulder_height_cm', 0):>7.1f} cm  ({meas.get('relative_to_shoulder_height_inches', 0):>5.1f} in)")
print("="*50)

# Display annotated image
print("\nAnnotated contact frame:")
display(IPImage(filename=out_image_path, width=800))

In [ ]:
#@title 3. Download Results
from google.colab import files as colab_files
import glob

output_dir = "/content/output"
png_files = glob.glob(os.path.join(output_dir, "contact_analyzed_*.png"))
csv_files = glob.glob(os.path.join(output_dir, "measurements_*.csv"))

print("Downloading files...")
for f in png_files + csv_files:
    print(f"  {os.path.basename(f)}")
    colab_files.download(f)